In [2]:
# -*- coding: utf-8 -*-
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import ht
import CoolProp as cp
import fluids
from IPython.display import Latex
from scipy.optimize import brentq


Un purificador de agua simple consiste en un recipiente a presión que se llena parcialmente con agua no potable, como se muestra en la figura. Un elemento calentador de cobre pulido se encuentra sumergido en el agua y causa evaporación. El calentador tiene un largo $L_{htr}=8$cm de largo y $W=10$cm de ancho (en la dirección normal a la hoja). 

[<img src="purificador.png" width="350"/>](purificador.png)

Una placa fría se fija a un lado del recipiente y se enfría a una temperatura uniforme de $T_{s,cp}=40ºC$. El vapor de agua condensa sobre la placa fría y se colecta como agua potable. El largo de la placa fría es de $L_{cp}=4$cm y su ancho es $W=10$cm. 

Ignore el efecto del aire no condensable en el recipiente.

     1. Determine el calor que debe entregarse al evaporador para poder recoger 3 kilogramos de agua por hora. Considere condiciones estacionarias.
    2. Determinar la temperatura del elemento calentador.
    3. Estimar el flujo crítico de calor y comparar con el flujo de calor que se entrega.



In [3]:
Tsat = cp.CoolProp.PropsSI('T','P',101.3e3,'Q',1,'Water')


In [40]:
fluid1 = 'Water'
Tw_cond = 40+273
L_cond = 4e-2
W_cond = 10e-2
L_eb = 8e-2
W_eb=10e-2
A_cond = L_cond*W_cond
A_eb = L_eb*W_eb
#m_cond = 3
masa_vapor_ref = 3.
#Q_eb = 4e3
#psat = 601e3
#psat = 88.3e3
psat=101.3e3  #primera iteracion
Tsat = cp.CoolProp.PropsSI('T','P',psat,'Q',1,'Water')
Tref = (Tsat + Tw_cond)/2
rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)

hfg_1 = (h_1_v-h_1_l)


hcond =ht.condensation.Nusselt_laminar(Tsat,Tw_cond,rho_v,rho_l,kl,mul,hfg_1,L_cond)

Qcond = hcond*L_cond*W_cond*(Tsat-Tw_cond)
mvap = Qcond/hfg_1
print('masa de condensado: %.2f kg/hr'%(mvap*3600))


def condicion_condensado(psat):
    masavapor = 3.
    Tsat = cp.CoolProp.PropsSI('T','P',psat,'Q',1,'Water')
    Tref = (Tsat + Tw_cond)/2
    rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
    rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
    cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

    h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
    h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
    kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
    mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)

    hfg_1 = (h_1_v-h_1_l)


    hcond =ht.condensation.Nusselt_laminar(Tsat,Tw_cond,rho_v,rho_l,kl,mul,hfg_1,L_cond)

    Qcond = hcond*L_cond*W_cond*(Tsat-Tw_cond)
    mvap = Qcond/hfg_1
    return (mvap*3600-masavapor)


masa de condensado: 3.22 kg/hr


In [41]:
Qcond

2015.5432927674353

In [42]:
psat = brentq(condicion_condensado,60e3,700e3)
Tsat = cp.CoolProp.PropsSI('T','P',psat,'Q',1,'Water')
Tref = (Tsat + Tw_cond)/2
rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)
sigma_l = cp.CoolProp.PropsSI('SURFACE_TENSION','T',Tref,'Q',0,fluid1)
hfg_1 = (h_1_v-h_1_l)


hcond = ht.condensation.Nusselt_laminar(Tsat,Tw_cond,rho_v,rho_l,kl,mul,hfg_1,L_cond)

Qcond = hcond*A_cond*(Tsat-Tw_cond)
Qeb = Qcond
mvap = Qcond/hfg_1
display(Latex('masa de condensado: %.2f kg/hr'%(mvap*3600)))
display(Latex(f'presion de saturacion: {psat/1e3:.2f}kPa'))
display(Latex(f'Temp de saturacion: {Tsat-273:.0f}ºC'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [43]:
Tw_eb = Tsat+42.5
Tref = (Tsat + Tw_eb)/2
rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)
sigma_l = cp.CoolProp.PropsSI('SURFACE_TENSION','T',Tref,'Q',0,fluid1)
hfg = h_1_v-h_1_l

In [44]:
Nu = hcond*L_cond/kl
Nu
g = 9.8
delta_cond = L_cond/Nu

In [45]:
h_eb = ht.boiling_nucleic.Rohsenow(rho_l,rho_v,mul,kl,cp_l,Tw_eb,hfg,sigma_l)
Qeb = h_eb*A_eb*(Tw_eb-Tsat)
mvap_eb = Qeb/hfg

In [46]:
deltaT_eb_h = Qeb / A_eb / h_eb
mvap_eb*3600

2.8677198997709166

In [47]:
def condicion_ebullicion(Tw_eb):
    
    Tref = (Tsat + Tw_eb)/2
    Te = Tw_eb - Tsat
    rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
    rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
    cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

    h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
    h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
    kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
    mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)
    sigma_l = cp.CoolProp.PropsSI('SURFACE_TENSION','T',Tref,'Q',0,fluid1)
    hfg = h_1_v-h_1_l
    h_eb = ht.boiling_nucleic.Rohsenow(rho_l,rho_v,mul,kl,cp_l,Tw_eb,hfg,sigma_l,Te)
    Qeb = h_eb*A_eb*(Tw_eb-Tsat)
    mvap = Qeb/hfg
    return (mvap*3600-masa_vapor_ref)
def data_ebullicion(Tw_eb):
    Tref = (Tsat + Tw_eb)/2
    Te = Tw_eb - Tsat
    rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
    rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
    cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

    h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
    h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
    kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
    mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)
    sigma_l = cp.CoolProp.PropsSI('SURFACE_TENSION','T',Tref,'Q',0,fluid1)
    hfg = h_1_v-h_1_l
    h_eb = ht.boiling_nucleic.Rohsenow(rho_l,rho_v,mul,kl,cp_l,Tw_eb,hfg,sigma_l,Te)
    Qeb = h_eb*A_eb*(Tw_eb-Tsat)
    mvap = Qeb/hfg
    return mvap/3600 -3


def out_ebullicion(Tw_eb):
    Tref = (Tsat + Tw_eb)/2
    Te = Tw_eb - Tsat
    rho_v = cp.CoolProp.PropsSI('D','T',Tsat,'Q',1,fluid1)
    rho_l = cp.CoolProp.PropsSI('D','T',Tref,'Q',0,fluid1)
    cp_l = cp.CoolProp.PropsSI('C','T',Tref,'Q',1,fluid1)

    h_1_v = cp.CoolProp.PropsSI('H','T',Tsat,'Q',1,fluid1)
    h_1_l = cp.CoolProp.PropsSI('H','T',Tsat,'Q',0,fluid1)
    kl = cp.CoolProp.PropsSI('CONDUCTIVITY','T',Tref,'Q',0,fluid1)
    mul = cp.CoolProp.PropsSI('VISCOSITY','T',Tref,'Q',0,fluid1)
    sigma_l = cp.CoolProp.PropsSI('SURFACE_TENSION','T',Tref,'Q',0,fluid1)
    hfg = h_1_v-h_1_l
    h_eb = ht.boiling_nucleic.Rohsenow(rho_l,rho_v,mul,kl,cp_l,Tw_eb,hfg,sigma_l,Te)
    Qeb = h_eb*A_eb*(Tw_eb-Tsat)
    mvap = Qeb/hfg
    return mvap,Qeb, Tw_eb

In [48]:
Tw_eb  = brentq(condicion_ebullicion,Tsat+10,Tsat+70)
mvap,Qeb, Tw_eb = out_ebullicion(Tw_eb)
display(Latex(f'Temp de pared en ebullicion: {Tw_eb-273:.0f}ºC'))
display(Latex(f'Salto Temp de pared: {Tw_eb-Tsat:.0f}ºC'))

display(Latex(f'Flujo de Calor: {Qeb/1e3:.1f}kW'))
display(Latex(f'masa vapor: {mvap*3600:.1f}kg/h'))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [54]:
Qcond,Qeb

(1889.9764189089121, 1889.9764189088658)

In [55]:
L_cap = (sigma_l/(rho_l-rho_v)*g)**0.5

Cmax = 0.15*12*np.pi*L_cap**2/A_eb

L_a = L_eb/L_cap

qmax = Cmax*hfg*(sigma_l*rho_v**2*(rho_l-rho_v)*g)**(1/4)
Qmax = qmax*A_eb
display(Latex(f'Flujo Máximo de Calor: {Qmax/1e3:.0f}kW'))

<IPython.core.display.Latex object>

In [56]:
qmax/1e6

3.1590338505834556

In [52]:
q_eb = Qeb/A_eb
q_eb/1e6

0.2362470523636082